<H1> <b>Capstone Project Data Science</b></H1> 

In [1]:
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [3]:
import scrapy
from scrapy.crawler import CrawlerProcess

In [4]:
from urllib.request import urlopen
from bs4 import BeautifulSoup


In [5]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)

In [6]:
soup = BeautifulSoup(html, 'lxml')
type(soup)

bs4.BeautifulSoup

<h3>Get Data from wikipedia using BeautifulSoup</h3>

In [77]:
My_table = soup.find('table',{'class':'wikitable sortable'})
Columns_title=My_table.findAll("th")
i=0
df=pd.DataFrame(columns=['Postcode', 'Borough', 'Neighbourhood'])
for row in My_table.findAll("tr"):
    cells = row.findAll("td")
    if len(cells) == 3:
        df.loc[len(df)+i]=[cells[0].find(text=True),cells[1].find(text=True),cells[2].find(text=True)]
        i=i+1 
        
           
            

<h3>Change type of DataFrame df to string</h3>

In [78]:
df=df.astype(str)
df.columns

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')

<h3> Select data where Borough different Not assigned </h3>

In [79]:
df=df.loc[df['Borough']!='Not assigned']

<h3>Replace all Neighbourhood=Not assigned by the value of Borough</h3>

In [80]:
df['Neighbourhood']=np.where(str(df['Neighbourhood'])=='Not assigned',df['Neighbourhood'] ,df['Borough'])
df.head(3)

,Postcode,Borough,Neighbourhood
4,M3A,North York,North York
6,M4A,North York,North York
8,M5A,Downtown Toronto,Downtown Toronto


<h3>Duplicate Postcode </h3>

In [81]:
df=df.groupby(['Postcode','Borough'])['Neighbourhood'].agg(','.join).reset_index()
df['Neighbourhood']=df['Neighbourhood'].str.replace('\n','')
df.head(3)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Scarborough,Scarborough"
1,M1C,Scarborough,"Scarborough,Scarborough,Scarborough"
2,M1E,Scarborough,"Scarborough,Scarborough,Scarborough"


<h3>Shape of data</h3>

In [82]:
df.shape

(103, 3)

<h3>Get geographical coordinates  for each postcode</h3>

In [83]:
Geospatial=pd.read_csv('C:/Users/pc/Downloads/Geospatial_Coordinates.csv' ,error_bad_lines=False)

In [84]:
Geospatial.head()
Geospatial.rename(columns={'Postal Code':'Postcode' },inplace=True)
Geospatial.head(3)

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711


In [85]:
df=df.join(Geospatial.set_index('Postcode'), on='Postcode')
df.head(3)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Scarborough,Scarborough",43.806686,-79.194353
1,M1C,Scarborough,"Scarborough,Scarborough,Scarborough",43.784535,-79.160497
2,M1E,Scarborough,"Scarborough,Scarborough,Scarborough",43.763573,-79.188711


<h2>Explore and cluster the neighborhood</h2>

<h3> filter data </h3>

In [87]:
df=df.loc[df['Borough'].str.contains('Toronto')]
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,East Toronto,43.676357,-79.293031
41,M4K,East Toronto,"East Toronto,East Toronto",43.679557,-79.352188
42,M4L,East Toronto,"East Toronto,East Toronto",43.668999,-79.315572
43,M4M,East Toronto,East Toronto,43.659526,-79.340923
44,M4N,Central Toronto,Central Toronto,43.728020,-79.388790


<h3>Cluster the neighborhood</h3>

In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

<h3>Group data by  neighborhood</h3>

In [110]:
toronto_grouped =df.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Latitude,Longitude
0,Central Toronto,43.714435,-79.398078
1,"Central Toronto,Central Toronto",43.693261,-79.397234
2,"Central Toronto,Central Toronto,Central Toronto",43.672710,-79.405678
3,"Central Toronto,Central Toronto,Central Toront...",43.686412,-79.400049
4,Downtown Toronto,43.659131,-79.382704


In [111]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 4, 4, 4, 0, 0, 0, 0, 2, 2])

<h3>Create Map of Cluster</h3>

In [112]:
# add clustering labels
toronto_grouped.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_grouped


,Cluster Labels,Neighbourhood,Latitude,Longitude
0,3,Central Toronto,43.714435,-79.398078
1,4,"Central Toronto,Central Toronto",43.693261,-79.397234
2,4,"Central Toronto,Central Toronto,Central Toronto",43.672710,-79.405678
3,4,"Central Toronto,Central Toronto,Central Toront...",43.686412,-79.400049
4,0,Downtown Toronto,43.659131,-79.382704
5,0,"Downtown Toronto,Downtown Toronto",43.655271,-79.381723
6,0,"Downtown Toronto,Downtown Toronto,Downtown Tor...",43.648198,-79.388790
7,0,"Downtown Toronto,Downtown Toronto,Downtown Tor...",43.628947,-79.394420
8,2,East Toronto,43.666209,-79.318504
9,2,"East Toronto,East Toronto",43.674278,-79.333880


<h3>Create map</h3>

In [119]:

map_clusters = folium.Map(location=[toronto_grouped['Latitude'].mean(),toronto_grouped['Longitude'].mean()], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_grouped['Latitude'],toronto_grouped['Longitude'],toronto_grouped['Neighbourhood'],toronto_grouped['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
toronto_merged =df

#merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged =toronto_merged.join(toronto_grouped.set_index('Neighbourhood'), on='Neighbourhood')

manhattan_merged.head() # check the last columns!